In [1]:
# Import modules
import tensorflow as tf

from src.modules.data_loader import create_ffn_dataset
from src.modules.training_utils import train_ffn
from src.models.fnn import build_ffn
from src.various.postprocess import plot_loss_curve

2025-01-31 17:03:11.576320: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-31 17:03:11.614502: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-31 17:03:11.615119: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-31 17:03:12.342824: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# File paths
tfrecord_file = "../data/FFNs/FFNs_init.tfrecord"

# Input parameters
max_atoms = 7662 # Required for fixed sized FNN
input_geometry_shape = (max_atoms, 3)  # Example geometry shape
input_frequency_shape = (1,)

# Training hyperparameters
batch_size = 32
epochs = 1
learning_rate = 1e-3

In [3]:
# Load dataset
print("Loading dataset...")
train_dataset, val_dataset, test_dataset = create_ffn_dataset(tfrecord_file)
print("Dataset loaded.")


Loading dataset...
📊 Total Samples: 25000, Train: 20000, Val: 2500, Test: 2500
📊 Final Split: Train=625, Validation=79, Test=79
Dataset loaded.


In [5]:
# Build model
print("Building model...")
model = build_ffn(input_geometry_shape, input_frequency_shape)
model.summary()

Building model...
Model: "GraphAIne_FFN_Model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 geometry_input (InputLayer  [(None, 7662, 3)]            0         []                            
 )                                                                                                
                                                                                                  
 conv1d (Conv1D)             (None, 7662, 64)             640       ['geometry_input[0][0]']      
                                                                                                  
 frequency_input (InputLaye  [(None, 1)]                  0         []                            
 r)                                                                                               
                                                              

In [6]:
# Train model
print("Training model...")
train_ffn(model, train_dataset, val_dataset, epochs=epochs, learning_rate=learning_rate, checkpoint_path="../checkpoints/ffn")

model.save("../checkpoints/ffn_model.keras")  # Save in new Keras format
print("✅ Model saved!")

Training model...

🚀 Epoch 1/1 - Training GraphAIne_FFN_Model
📊 Epoch 1 - Train Loss: 7401.9833984375 | Val Loss: 0.2562119662761688
💾 Checkpoint saved at ../checkpoints/ffn/ckpt-1

✅ Training Complete!
✅ Model saved!


In [ ]:
# Plot loss (train + validation) vs. epochs
plot_loss_curve("training_log.csv")

In [7]:
# Evaluate test loss
test_loss = tf.keras.metrics.Mean(name="test_loss")

for test_inputs, test_labels  in test_dataset:
    test_predictions = model([test_inputs['geometries'], test_inputs['frequency']], training=False)
    test_loss.update_state(model.loss(test_labels,test_predictions))

print(f"\n📊 Test loss: {test_loss.result().numpy()}")


📊 Test loss: 0.22950375080108643


In [ ]:
#################################################################

In [ ]:
############################# TESTS #############################

In [ ]:
#################################################################

In [13]:
import tensorflow.keras.backend as K

get_frequency_output = K.function([model.input], [model.get_layer("dense_1").output])
frequency_test_output = get_frequency_output([test_inputs['geometries'], test_inputs['frequency']])

print("Frequency Branch Output:", frequency_test_output)

Frequency Branch Output: [array([[0.        , 0.        , 0.05055495, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.14202367, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.10424311, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.0366358 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.05254341, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.0326589 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)]


In [ ]:
for layer in model.layers:
    if "geometry_input" in layer.name or "dense" in layer.name:
        print(layer.name, "weights:", layer.get_weights())

In [11]:
for layer in model.layers:
    if "frequency_input" in layer.name or "dense" in layer.name:
        print(layer.name, "weights:", layer.get_weights())

frequency_input weights: []
dense_1 weights: [array([[-0.03125258, -0.19616944,  0.19884504, -0.21459624, -0.13259822,
         0.12743568, -0.1098198 , -0.1495179 , -0.06856835, -0.07372682,
        -0.04990563, -0.16591886,  0.15853071, -0.01603162,  0.1616533 ,
        -0.18770212, -0.19912046,  0.08992108,  0.09111468, -0.13747929,
        -0.05866259,  0.09282053, -0.03248699, -0.03932819, -0.18423969,
         0.22208825, -0.1217444 ,  0.06571238, -0.09813835,  0.12226506,
         0.11363782, -0.04195024, -0.06648743, -0.15892968, -0.11510065,
         0.05735431, -0.13043344, -0.00275537,  0.02114777,  0.03570646,
         0.10901417,  0.09401481,  0.19669701, -0.14399286,  0.06650866,
        -0.00122782, -0.1556722 , -0.07437253,  0.18812664,  0.09837634,
        -0.14696275,  0.11794274, -0.17836855,  0.18163513,  0.01577363,
         0.05548303, -0.17710581,  0.20523319,  0.06843634, -0.05871236,
         0.16309506,  0.05781835, -0.13222143, -0.20251147, -0.03364436,
     

In [ ]:
for layer in model.layers:
    print(layer.name, "input shape:", layer.input_shape, "output shape:", layer.output_shape)

In [9]:
import numpy as np

for test_inputs, test_labels in test_dataset.take(1):  # Only one batch
    real_geometry = test_inputs['geometries']  # Extract geometry
    frequency_input = test_inputs['frequency']  # Extract frequency

    # Create a dummy geometry (same shape but filled with zeros)
    dummy_geometry = np.zeros_like(real_geometry)

    # Predict with real geometry
    charges_with_real_geometry = model([real_geometry, frequency_input], training=False)

    # Predict with dummy geometry
    charges_with_dummy_geometry = model([dummy_geometry, frequency_input], training=False)

    # Print results
    print("Real Geometry Charges:\n", charges_with_real_geometry.numpy())
    print("\nDummy Geometry Charges:\n", charges_with_dummy_geometry.numpy())

    # Compute difference between outputs
    diff = np.abs(charges_with_real_geometry.numpy() - charges_with_dummy_geometry.numpy())
    print("\nDifference Between Predictions:\n", diff)
    
    # Check if the predictions are too similar
    if np.allclose(charges_with_real_geometry.numpy(), charges_with_dummy_geometry.numpy(), atol=1e-5):
        print("\n🚨 WARNING: The model is ignoring geometry! 🚨")
    else:
        print("\n✅ Geometry is influencing the output! ✅")


Real Geometry Charges:
 [[[-0.01291188 -0.00435527 -0.00722631 -0.00238905 -0.01535218
    0.00706501]
  [-0.00356425 -0.02003371  0.0013345   0.00492781 -0.00840082
   -0.00448991]
  [ 0.00290506 -0.00548862  0.00647704  0.00770468  0.00784223
   -0.00195733]
  ...
  [-0.00724814  0.00098666 -0.00920929 -0.00778913 -0.00744226
   -0.00279231]
  [-0.00961921 -0.00798832 -0.00445379 -0.00569218 -0.00590705
    0.0078069 ]
  [-0.00612885 -0.00138874  0.00561029  0.00036992 -0.01207189
    0.00630799]]

 [[-0.01291188 -0.00435527 -0.00722631 -0.00238905 -0.01535218
    0.00706501]
  [-0.00356425 -0.02003371  0.0013345   0.00492781 -0.00840082
   -0.00448991]
  [ 0.00290506 -0.00548862  0.00647704  0.00770468  0.00784223
   -0.00195733]
  ...
  [-0.00724814  0.00098666 -0.00920929 -0.00778913 -0.00744226
   -0.00279231]
  [-0.00961921 -0.00798832 -0.00445379 -0.00569218 -0.00590705
    0.0078069 ]
  [-0.00612885 -0.00138874  0.00561029  0.00036992 -0.01207189
    0.00630799]]

 [[-0.012911

In [10]:
print("Min frequency:", np.min(test_inputs['frequency']))
print("Max frequency:", np.max(test_inputs['frequency']))


Min frequency: 0.02
Max frequency: 0.98
